## Introduction

The entire Tracking the Sun dataset contains over 1 million points of data for solar cell systems in the United States. In order to set up the dataset for plotting, the locations of the solar cell systems need to be geocoded. Using the geocoding service Nominatim in combination with geopy, the coordinates of each solar cell system will be found.

## Geocoding: Step by Step

First, the user will import the geocode python file to enable access to the functions to geocode a given dataset. The data file path needs to be specified and passed into the geocoding function. This notebook will walk the user through how to use the geocoding program.

In [1]:
import geocoding
from geocoding import read_data
from geocoding import find_unique_cities
from geocoding import create_save_files
from geocoding import geocode_unique
from geocoding import assign_lat_long_columns
from geocoding import geocode_full

In [3]:
data_file_path = 'Solar-PV/data/TTS_sample.csv'
data = read_data(data_file_path)
data

FileNotFoundError: [Errno 2] No such file or directory: 'Solar-PV/data/TTS_sample.csv'

Geocoding the entire dataset line by line would take over 7 days for 1 million datapoints. So, instead of doing this, only the unique locations are geocoded so that computing time/power isn't wasted on duplicate locations. In this example, the number of unique cities is about 100 less than the total number of cities in the original dataset, thus the amount of cities is geocode is reduced by a third. Using this method on the full dataset results in only about 18% of the datapoints needing to be geocoded.

The find_unique_cities function creates a datframe of the unique cities in the dataset and preps the locations for geocoding by creating a column 'city_state_country'. This relies on the the original dataframe haivng columns for the city and state and assumign all location are in the USA. The reason for this is that it makes using the geocoding service, Nominatim, more accurate. Without the country specificied there is a chance the geocoding will pick city with the same name in the wrong country.

In [ ]:
unique_cities = find_unique_cities(data)
unique_cities

If it's the user's first time running geocoding on their data, they should use the the create_save_files function to create blank csv files to store the geocoding progress as its runs. One file is used to store the geocoding data and one file is used to store cities that result in geocode errors. Many of these errors are due to typos in the city names in the original dataset. The purpose of collecting these is so the user can go back and fix the dataset manually after this code runs if they wish.

WARNING: Running this function after the geocoding is part of the way through will delete the progress. Only run this once before geocoding a new dataset.

In [ ]:
create_save_files()

The geocode_unique function is used to geocode all the unique cities in the original dataset and save the progress as it goes (through the geocode_save function) as well as the cities that result in errors. It takes the unique cities dataframe as an arguement and returns a dataframe with the unique cities geocoded. It also saves this as a .csv as well for reference.

Note: If the geocoding below is interrupted, the user can simply run the cell below again and the geocoding will pickup where it left off. (Again, do NOT run the create_save_files function at this stage)

In [ ]:
unique_geo_df = geocode_unique(unique_cities)

In [ ]:
unique_geo_df

Latitude and longitude coordinates are broken out into two seperate columns through the assign_lat_long_columns function. This makes analyzing and plotting the data easier down the line.

In [ ]:
unique_geo_df = assign_lat_long_columns(unique_geo_df)

In [ ]:
unique_geo_df

Finally, the full dataset is geocoded by running the geocode_full function which uses the dataframe of unique cities with latitude and longitude values to assing values to every location in the original dataframe.

In [ ]:
data = geocode_full(data, unique_geo_df)

In [ ]:
data[['city_state_country', 'latitude', 'longitude']]

## Geocoding: One Shot

To geocode the dataset without seeing the step by step resulsts run create_save files and geocode_go. The output is saved to 'TTS_fully_geocoded_sample.csv' which is a the original data with the latitude and longitude coordinates added for each location.

In [ ]:
import geocoding
from geocoding import create_save_files
from geocoding import geocode_go

In [ ]:
create_save_files()

In [ ]:
data = geocode_go('data/TTS_sample.csv')
data[['city_state_country', 'latitude', 'longitude']]

,city_state_country,latitude,longitude
0,"Washington D.C., DC, USA",38.895037,-77.036543
1,"OCEANSIDE, CA, USA",33.195870,-117.379483
2,"St. George, UT, USA",37.104153,-113.584131
3,"Narberth, PA, USA",40.008446,-75.260460
4,"OAKLAND, CA, USA",37.804456,-122.271356
...,...,...,...
304,"Etna, NH, USA",43.692849,-72.221756
305,"Falmouth, MA, USA",43.729525,-70.241993
306,"Naugatuck, CT, USA",41.486019,-73.050943
307,"Lititz, PA, USA",40.157125,-76.307210
